# Conversation Pattern
In the previous video, we used two-agent conversation, which can be started by the initiate_chat method. Two-agent chat is a useful conversation pattern but AutoGen offers more. 
In this video, we will first dig a little bit more into the two-agent chat pattern and chat result, then we will show you several conversation patterns that involve more than two agents.

Types:
1. Two-agent chat: the simplest form of conversation pattern where two agents chat with each other.
2. Sequential chat: a sequence of chats between two agents, chained together by a carryover mechanism, which brings the summary of the previous chat to the context of the next chat.
3. Group Chat: a single chat involving more than two agents. An important question in group chat is: What agent should be next to speak? To support different scenarios, here are the different ways to organize agents in a group chat:
    a. Support several strategies to select the next agent: round_robin, random, manual (human selection), and auto (Default, using an LLM to decide).
    b. Provide a way to constrain the selection of the next speaker.
    c. Pass in a function to customize the selection of the next speaker. With this feature, you can build a StateFlow model which allows a deterministic workflow among your agents.
4. Nested Chat: package a workflow into a single agent for reuse in a larger workflow.


# Setup

In [ ]:
import os

from dotenv import load_dotenv
load_dotenv()

azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT")
azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
api_key=os.getenv("AZURE_OPENAI_KEY")

llm_config = [{"model": azure_deployment,
               "api_type": "azure",
               "temperature": 0.9,
               "api_key": api_key,
               "base_url": azure_endpoint,
               "api_version": "2024-02-15-preview"}] 

# Sequential Chat

In this pattern, the pair of agents first start a two-agent chat, then the summary of the conversation becomes a <b>carryover</b> for the next two-agent chat. The next chat passes the carryover to the carryover parameter of the context to generate its initial message.

<b> Carryover accumulates as the conversation moves forward, so each subsequent chat starts with all the carryovers from previous chats.</b>

In [ ]:
from autogen import ConversableAgent

# The Number Agent always returns the same numbers.
number_agent = ConversableAgent(
    name="Number_Agent",
    system_message="You return me the numbers I give you, one number each line.",
    llm_config={"config_list": llm_config},
    human_input_mode="NEVER",
)

# The Adder Agent adds 1 to each number it receives.
adder_agent = ConversableAgent(
    name="Adder_Agent",
    system_message="You add 1 to each number I give you and return me the new numbers, one number each line.",
    llm_config={"config_list": llm_config},
    human_input_mode="NEVER",
)

# The Multiplier Agent multiplies each number it receives by 2.
multiplier_agent = ConversableAgent(
    name="Multiplier_Agent",
    system_message="You multiply each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config={"config_list": llm_config},
    human_input_mode="NEVER",
)

# The Subtracter Agent subtracts 1 from each number it receives.
subtracter_agent = ConversableAgent(
    name="Subtracter_Agent",
    system_message="You subtract 1 from each number I give you and return me the new numbers, one number each line.",
    llm_config={"config_list": llm_config},
    human_input_mode="NEVER",
)

# The Divider Agent divides each number it receives by 2.
divider_agent = ConversableAgent(
    name="Divider_Agent",
    system_message="You divide each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config={"config_list": llm_config},
    human_input_mode="NEVER",
)

In [ ]:
# Start a sequence of two-agent chats.
# Each element in the list is a dictionary that specifies the arguments
# for the initiate_chat method.
max_turns = 1
chat_results = number_agent.initiate_chats(
    [
        {
            "recipient": adder_agent,
            "message": "14",
            "max_turns": max_turns,
            "summary_method": "last_msg",
        },
        {
            "recipient": multiplier_agent,
            "message": "These are my numbers",
            "max_turns": max_turns,
            "summary_method": "last_msg",
        },
        {
            "recipient": subtracter_agent,
            "message": "These are my numbers",
            "max_turns": max_turns,
            "summary_method": "last_msg",
        },
        {
            "recipient": divider_agent,
            "message": "These are my numbers",
            "max_turns": max_turns,
            "summary_method": "last_msg",
        },
    ]
)

In [ ]:
print("First Chat Summary: ", chat_results[0].summary)
print("Second Chat Summary: ", chat_results[1].summary)
print("Third Chat Summary: ", chat_results[2].summary)
print("Fourth Chat Summary: ", chat_results[3].summary)

# Group Chat

Group chat involves more than two agents. In group chat all agents contribute to a single conversation thread and share the same context. This is useful for tasks that require collaboration among multiple agents.

A group chat is orchestrated by a special agent type GroupChatManager. In the first step of the group chat, the Group Chat Manager selects an agent to speak. Then, the selected agent speaks and the message is sent back to the Group Chat Manager, who broadcasts the message to all other agents in the group. This process repeats until the conversation stops.

The Group Chat Manager can use several strategies to select the next agent. Currently, the following strategies are supported:

1. round_robin: The Group Chat Manager selects agents in a round-robin fashion based on the order of the agents provided.
2. random: The Group Chat Manager selects agents randomly.
3. manual: The Group Chat Manager selects agents by asking for human input.
4. auto: The default strategy, which selects agents using the Group Chat Manager’s LLM.


It can be hard to control if the number of participating agents is large. AutoGen provides a way to constrain the selection of the next speaker by using the allowed_or_disallowed_speaker_transitions argument of the GroupChat class.

The allowed_or_disallowed_speaker_transitions argument is a dictionary that maps a given agent to a list of agents that can (or cannot) be selected to speak next. The speaker_transitions_type argument specifies whether the transitions are allowed or disallowed.

In [ ]:
from autogen import ConversableAgent

# The Number Agent always returns the same numbers.
number_agent = ConversableAgent(
    name="Number_Agent",
    system_message="You return the number given to you.",
    llm_config={"config_list": llm_config},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg["content"] or "achieved" in msg["content"]
)

# The Adder Agent adds 1 to each number it receives.
adder_agent = ConversableAgent(
    name="Adder_Agent",
    system_message="You add 1 to number provided to you and return the number. You will not perform other computations other than that.",
    description = "Add 1 to each input number.",
    llm_config={"config_list": llm_config},
    human_input_mode="NEVER",
)

# The Multiplier Agent multiplies each number it receives by 2.
multiplier_agent = ConversableAgent(
    name="Multiplier_Agent",
    system_message="You multiply number provided to you by 2 and return the number. You will not perform other computations other than that.",
    description = "Multiply input number by 2.",
    llm_config={"config_list": llm_config},
    human_input_mode="NEVER",
)

# The Subtracter Agent subtracts 1 from each number it receives.
subtracter_agent = ConversableAgent(
    name="Subtracter_Agent",
    system_message="You subtract 1 from number provided to you and return the number. You will not perform other computations other than that.",
    description = "Subtract 1 from input number.",
    llm_config={"config_list": llm_config},
    human_input_mode="NEVER",
)

# The Divider Agent divides each number it receives by 2.
divider_agent = ConversableAgent(
    name="Divider_Agent",
    system_message="You divide number provided to you by 2 and return the number. You will not perform other computations other than that.",
    description = "Divide input number by 2.",
    llm_config={"config_list": llm_config},
    human_input_mode="NEVER",
)

allowed_transitions = {
    number_agent: [adder_agent, number_agent],
    adder_agent: [multiplier_agent, number_agent],
    subtracter_agent: [divider_agent, number_agent],
    multiplier_agent: [subtracter_agent, number_agent],
    divider_agent: [adder_agent, number_agent],
}

In [ ]:
from autogen import GroupChat
from autogen import GroupChatManager

group_chat = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent, divider_agent, number_agent],
    messages=[],
    max_round=50,#max iterations of the group chat
    allowed_or_disallowed_speaker_transitions=allowed_transitions,
    speaker_transitions_type="allowed",
    speaker_selection_method="auto",
    #send_introductions=True #Sometimes it is useful have each agent introduce themselves to other agents in the group chat.
)

group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config={"config_list": llm_config},
)

# Number Agent to start a two-agent chat with the Group Chat Manager, which runs the group chat internally. 
# Group Chat Manager terminates the two-agent chat when the internal group chat is done. 
# Because the Number Agent is selected to speak by us, it counts as the first round of the group chat.
chat_result = number_agent.initiate_chat(
    group_chat_manager,
    message="My number is 3, use operations to turn this to 15. Terminate when 15 is achieved.",
    summary_method="reflection_with_llm",
)